# 02 - Create the voting token

**Description**: the purpose of this notebook is to create the senate's coin that will be distributed to each of the member of the senate when a voting proposal pops up and the member wish to participate. The proposer is forced to deposit part of its stake to issue a proposal (in order to avoid spam, we would need to make this process costly). The voters (members of the senate) would need to spend some coins for voting, part of which would go to the validators as a transaction fee, and part would remain to the voting issuer as a contribution for partecipating into the ecosystem.

### 00 - Set-up

In [2]:
# Make sure that the algorand-sdk is installed before executing the following jupyter notebook
!pip install py-algorand-sdk

### 01 - Imports
Import the all the required methods and libraries to properly run this notebook 

In [1]:
# Add "algo_util.py" to the list of available dependencies of the project
import sys, os, json
from pathlib import Path

codepath = (Path( os.curdir ) / "shared").resolve()
sys.path.append(str(codepath))

In [2]:
# loading credential utility
from algo_util import load_credentials 

# algo-sdk utilities
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error

### 02 - Load parameters

In [3]:
# load credentials from storage
credentials = load_credentials(file_name = "credentials_temp")

print(f"{len(credentials['members_credentials'])} members' credentials loaded from storage")
print(f"Loading also senate account credentials: {credentials['usiv']['public']}")

32 members' credentials loaded from storage
Loading also senate account credentials: FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI


In [4]:
# load the account
usiv = credentials["usiv"]
print(usiv["public"])

FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI


## 03 - Create the USIV token (ASA -> Algorand Standard Asset)
This is our first token. The account USIV will create an asset called `USIV Coin`. 

During creation, we need to assign the following roles, which we will use later ...

* **Manager:** can change the thre follwoing roles
* **Reserve:** where not yet distributed assets reside
* **Freeze:** can freeze assets (e.g. to wait for KYC)
* **Clawback:** can undo transactions *if users have opted in*

See https://developer.algorand.org/docs/features/asa/ 

In [5]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(
    algod_token='', 
    algod_address=credentials['algod_test'], 
    headers=credentials['purestake_token']
)

### 03.1 - Setup the token configuration
N.B:
* Remember that tokens are divisible
* We choose 2 decimals
    * Our token can be divided into units as small as $\frac{1}{10^2} = \frac{1}{100} = 0.01$
* To create 1000 tokens, we must create $1000 \cdot 10^2$ *small units*
* we also need to define the token specifics (name, unit, url, etc....)

In [6]:
sp = algod_client.suggested_params()

In [7]:
sp = algod_client.suggested_params()

# token parameters
token_supply    = 1000                                   # Token supply (big units)
token_decimals  = 0                                     # Digits after the comma
token_total     = token_supply * 10**token_decimals     # Specify number of SMALLER unit ("cents")

token_name      = "USIV Coin"                           
token_url       = "https://usi.ch"                     
token_unit      = "USIV"                                # Abbreviation, e.g. shown in Algorand Wallet app  

### 03.2 - Create the asset ('asset creation transaction')

In [8]:
txn = AssetConfigTxn(
    sender=usiv['public'],                      # Creator of the ASA
    sp=sp,                      
    total=token_total,                         # Total supply in SMALL unit
    decimals=token_decimals,                   # Decimals
    default_frozen=False,                      # Are tokens frozen by default?
    unit_name=token_unit,                      # Abbreviation     
    asset_name=token_name,                     # Name
    url=token_url,                             # URL
    manager=usiv['public'],                     # Special roles
    reserve=usiv['public'],                     # Special roles
    freeze=usiv['public'],                      # Special roles
    clawback=usiv['public']                     # Special roles
)
print(txn)

{'sender': 'FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI', 'fee': 1000, 'first_valid_round': 41168408, 'last_valid_round': 41169408, 'note': None, 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'acfg', 'rekey_to': None, 'index': 0, 'total': 1000, 'default_frozen': False, 'unit_name': 'USIV', 'asset_name': 'USIV Coin', 'manager': 'FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI', 'reserve': 'FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI', 'freeze': 'FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI', 'clawback': 'FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI', 'url': 'https://usi.ch', 'metadata_hash': None, 'decimals': 0}


### 03.3 - Sign and send

In [9]:
stxn = txn.sign(usiv['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

WTWY772IMFOOQGGRF4NOETQY72VBDGCSCNEMKWJTKJU7JI5RUELA


### 03.4 - Wait for confirmation

In [10]:
# Wait for the transaction to be confirmed
from algo_util import wait_for_confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  41168409.
Waiting for round 41168409 to finish.
Waiting for round 41168410 to finish.
Transaction WTWY772IMFOOQGGRF4NOETQY72VBDGCSCNEMKWJTKJU7JI5RUELA confirmed in round 41168411.


### 03.5 - Asset statistics

First, we need to memorize the index of the coin

In [11]:
# Get the asset ID and open in Algoexplorer
USIV_id = txinfo['asset-index']
print(USIV_id)
print(f'https://testnet.algoexplorer.io/asset/{USIV_id}')

685785113
https://testnet.algoexplorer.io/asset/685785113


In [12]:
# Path to the newly generate credential (temporary file)
credential_path = (Path( os.curdir ) / ".." / "assets" / "token" / "token_info" ).resolve()

# dictionary containing info about the token
token_info = {
    "index": USIV_id,
    "name": token_name,
    "unit": token_unit,
    "supply": token_supply,
    "decimals": token_decimals
}

with open(credential_path, 'w') as outfile:
    outfile.write(json.dumps(token_info))

In [13]:
# check account info
account_info = algod_client.account_info(usiv["public"])
print(account_info)

{'address': 'FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI', 'amount': 19746000, 'amount-without-pending-rewards': 19746000, 'apps-local-state': [], 'apps-total-schema': {'num-byte-slice': 52, 'num-uint': 130}, 'assets': [{'amount': 0, 'asset-id': 683766647, 'is-frozen': False}, {'amount': 998, 'asset-id': 684625427, 'is-frozen': False}, {'amount': 1000, 'asset-id': 685785113, 'is-frozen': False}], 'created-apps': [{'id': 683790887, 'params': {'approval-program': 'BSADAAEgJggKVG90YWxWb3RlcwhJbiBmYXZvcgdBZ2FpbnN0B1ZlcmRpY3QKVG90YWxPcHRJbghIYXNWb3RlZANZZXMDVGllMRgiEkAA5DEZIxJAAMUxGSISQAAnMRmBAhJAAB0xGYEEEkAADzEZgQUSQAABADEAMgkSQzEAMgkSQyNDKGQkDkQxAoHuhM0TDDEEge7+zhMNEUAAfCInBWInBhJAAHA2GgA2GgBkIwhnMQAnBScGZigoZCMIZyhkJBJAAAIjQylkKmQNQAA5KmQpZA1AACIpZCpkEkAAEyNAAAEAgAZXaW5uZXInB2dC/9MrJwdnQv/MK4AIUmVqZWN0ZWRnQv+9K4AIQXBwcm92ZWRnQv+uIkMiQycEZCQORCInBYACTm9mJwQnBGQjCGcjQzEAgCAsMVXjH9rx/W7D1z2qjkzlLA4XAOf/XhNf+kcBDE3evRJEgAhWb3RlSW5mb4A1Rm9yIGVhY2ggcHJvcG9zYWwsIHZvdGUgaW4gZmF2b3